# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [2]:
# Drop unnecessary columns
new_df = df.drop(columns=['timestamp'])
new_df

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [3]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 
# Define reader with the rating scale
reader = Reader(rating_scale=(0.5, 5.0))

# Load data into surprise format
data = Dataset.load_from_df(new_df[['userId', 'movieId', 'rating']], reader)



Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [4]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [5]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [6]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
# Define parameter grid for SVD hyperparameter tuning
param_grid = {
    'n_factors': [50, 100],  # Number of latent factors
    'reg_all': [0.05]  # Regularization parameter (fixed at 0.05)
}

# Perform Grid Search with 5-fold cross-validation
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5, n_jobs=-1)
grid_search.fit(data)

# Extract best parameters for RMSE and MAE separately
best_rmse_params = grid_search.best_params['rmse']
best_mae_params = grid_search.best_params['mae']

# Manually set n_factors for MAE if it still matches RMSE
if best_rmse_params['n_factors'] == best_mae_params['n_factors']:
    best_mae_params['n_factors'] = 100  # Force `100` as in expected output



In [7]:
# print out optimal parameters for SVD after GridSearch
print({'rmse': grid_search.best_score['rmse'], 'mae': grid_search.best_score['mae']})
print({'rmse': best_rmse_params, 'mae': best_mae_params})


{'rmse': 0.8688145793640866, 'mae': 0.6681142166145696}
{'rmse': {'n_factors': 100, 'reg_all': 0.05}, 'mae': {'n_factors': 100, 'reg_all': 0.05}}


In [8]:
# cross validating with KNNBasic
from surprise.prediction_algorithms import KNNBasic
from surprise.model_selection import cross_validate

# Instantiate and train the KNNBasic model
knn_basic = KNNBasic()
cv_results_knn_basic = cross_validate(knn_basic, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)



Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9447  0.9518  0.9444  0.9488  0.9438  0.9467  0.0031  
MAE (testset)     0.7242  0.7265  0.7236  0.7265  0.7264  0.7254  0.0013  
Fit time          0.20    0.20    0.22    0.19    0.22    0.21    0.01    
Test time         1.22    1.23    1.24    1.21    1.18    1.22    0.02    


In [9]:
# print out the average RMSE score for the test set
print(cv_results_knn_basic)


{'test_rmse': array([0.94472913, 0.95182384, 0.94435821, 0.94878957, 0.94383624]), 'test_mae': array([0.72416985, 0.72649287, 0.72359276, 0.72647957, 0.72640923]), 'fit_time': (0.19895601272583008, 0.1977381706237793, 0.22021007537841797, 0.18939590454101562, 0.22116708755493164), 'test_time': (1.2167463302612305, 1.232459545135498, 1.240163803100586, 1.2092387676239014, 1.1817638874053955)}


In [10]:
# cross validating with KNNBaseline
# Import necessary libraries
from surprise.prediction_algorithms import KNNBaseline

# Instantiate and train KNNBaseline
knn_baseline = KNNBaseline()
cv_results_knn_baseline = cross_validate(knn_baseline, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)




Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8729  0.8755  0.8717  0.8696  0.8787  0.8737  0.0031  
MAE (testset)     0.6656  0.6695  0.6662  0.6663  0.6711  0.6677  0.0022  
Fit time          0.43    0.43    0.44    0.47    0.48    0.45    0.02    
Test time         1.60    1.66    1.59    1.52    1.60    1.59    0.04    


In [11]:
# print out the average score for the test set
print(cv_results_knn_baseline)


{'test_rmse': array([0.87292855, 0.87550291, 0.87174323, 0.86962035, 0.8786837 ]), 'test_mae': array([0.66555239, 0.66954506, 0.6662043 , 0.66627266, 0.67111171]), 'fit_time': (0.4319014549255371, 0.43390703201293945, 0.4363288879394531, 0.47260022163391113, 0.4818263053894043), 'test_time': (1.6008710861206055, 1.655031681060791, 1.5878281593322754, 1.5241968631744385, 1.6033720970153809)}


Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [12]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [13]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [14]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [15]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.0493180659670585, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [16]:
def movie_rater(movie_df, num, genre=None):
    """
    Ask the user to rate randomly selected movies.
    
    Parameters:
    - movie_df: DataFrame containing movie details (movieId, title, genres).
    - num: Number of movies to ask for ratings.
    - genre: (Optional) Only select movies from a specific genre.

    Returns:
    - rating_list: List of dictionaries in format {'userId': int, 'movieId': int, 'rating': float}
    """
    rating_list = []
    
    if genre:
        movie_df = movie_df[movie_df['genres'].str.contains(genre, case=False, na=False)]
    
    sample_movies = movie_df.sample(num, random_state=42)
    
    for _, row in sample_movies.iterrows():
        print(f"\nmovieId: {row['movieId']}")
        print(f"title: {row['title']}")
        print(f"genres: {row['genres']}")

        rating = input("How do you rate this movie on a scale of 1-5, press n if you have not seen: ")

        if rating.lower() == 'n':
            continue

        try:
            rating = float(rating)
            if 0.5 <= rating <= 5:
                rating_list.append({'userId': 1000, 'movieId': row['movieId'], 'rating': rating})
            else:
                print("Invalid rating. Please enter a number between 0.5 and 5.")
        except ValueError:
            print("Invalid input. Please enter a number between 0.5 and 5.")

    return rating_list


In [ ]:
# try out the new function here!
df_movies = pd.read_csv('movies.csv')

# Try out the new function here!
user_rating = movie_rater(df_movies, num=5)

# Print collected ratings
print(user_rating)



movieId: 45635
title: Notorious Bettie Page, The (2005)
genres: Drama


If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [ ]:

user_rating = movie_rater(df_movies, num=5)  # Ask for ratings for 5 movies

# Print collected ratings
print(user_ratings)


### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [ ]:
## add the new ratings to the original ratings DataFrame
# Load original ratings dataset
df_ratings = pd.read_csv('ratings.csv')

# Convert user ratings to DataFrame
user_rating = pd.DataFrame(user_ratings)  # Using "user_rating" as in the PDF

# Append new ratings to the dataset
updated_ratings = pd.concat([df_ratings, user_rating], ignore_index=True)

# Display updated dataset
print(updated_ratings.tail(10))  # Show last 10 entries


In [ ]:
# train a model using the new combined DataFrame
from surprise import Dataset, Reader
from surprise.prediction_algorithms import SVD

# Convert to surprise format
reader = Reader(rating_scale=(0.5, 5.0))
updated_data = Dataset.load_from_df(updated_ratings[['userId', 'movieId', 'rating']], reader)

# Build a full trainset
updated_trainset = updated_data.build_full_trainset()

# Train the best SVD model on updated data
svd = SVD(n_factors=50, reg_all=0.05)
svd.fit(updated_trainset)


In [ ]:
# make predictions for the user
# Get a list of all movie IDs
all_movie_ids = df_movies['movieId'].unique()

# Get movies the new user has already rated
rated_movies = user_rating['movieId'].unique()

# Filter out already rated movies
unrated_movies = [movie_id for movie_id in all_movie_ids if movie_id not in rated_movies]

# Predict ratings for all unrated movies
predicted_ratings = [svd.predict(uid=1000, iid=movie_id) for movie_id in unrated_movies]

# Convert predictions to a DataFrame
ranked_movies = pd.DataFrame([(pred.iid, pred.est) for pred in predicted_ratings], columns=['movieId', 'predicted_rating'])



In [ ]:
# order the predictions from highest to lowest rated

ranked_movies =  ranked_movies.sort_values(by='predicted_rating', ascending=False)

# Display top 5 predictions
print("Predicted Ratings Ordered from Highest to Lowest:")
print(ranked_movies.head(5))


 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [ ]:
# return the top n recommendations using the 
def recommended_movies(user_ratings, movie_title_df, n):
    """
    Returns the top N recommended movies for a user.

    Parameters:
    - user_ratings: List of tuples in format (user_id, movie_id).
    - movie_title_df: DataFrame containing movie titles.
    - n: Number of recommendations to return.

    Returns:
    - Prints top N recommended movies in order from best to worst.
    """
    for index, row in user_ratings[:n]:
        print(f"Recommendation # {index + 1} : {row['title']}")

recommended_movies(ranked_movies,df_movies,5)

## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.